In [1]:
import optuna
import pandas as pd
import numpy as np
import torch
import json
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error
import lightgbm as lgb

In [2]:
chemB_mol = torch.load('../ChemBERTa/chemberta_chromophore_embeddings_16k.pt', map_location="cpu")
chemB_sol = torch.load('../ChemBERTa/chemberta_solvent_embeddings_16k.pt', map_location="cpu")

print(chemB_mol.shape)
print(chemB_sol.shape)

C:\Users\kappa\AppData\Local\Temp\ipykernel_29556\3998124535.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  chemB_mol = torch.load('../ChemBERTa/chemberta_chromophore_e

(16648, 768)
(16648, 768)


In [3]:
molformer_mol = torch.load('../MolFormer/chromophores.pt', map_location="cpu")
molformer_sol = torch.load('../MolFormer/solvents.pt', map_location="cpu")

print(molformer_mol.shape)
print(molformer_sol.shape)

torch.Size([16648, 768])
torch.Size([16648, 768])


C:\Users\kappa\AppData\Local\Temp\ipykernel_29556\2716273118.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  molformer_mol = torch.load('../MolFormer/chromophores.pt', m

In [4]:
vecs_mols = torch.load("../torch_data/mols_new.pt", map_location="cpu")
print(vecs_mols.shape)

vecs_sols = torch.load("../torch_data/sols_new.pt", map_location="cpu")
print(vecs_sols.shape)

torch.Size([16648, 256])
torch.Size([16648, 256])


C:\Users\kappa\AppData\Local\Temp\ipykernel_29556\3264551377.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vecs_mols = torch.load("../torch_data/mols_new.pt", map_loca

In [33]:
msl_new = pd.read_csv('../input_data\msl_new.csv')
print(len(msl_new))
msl_new.head()

16648


,Chromophore,Solvent,Quantum yield
0,O=C([O-])c1ccccc1-c1c2ccc(=O)cc-2oc2cc([O-])ccc12,O,0.950
1,O=C([O-])c1ccccc1C1=c2cc3c4c(c2Oc2c1cc1c5c2CCC...,CO,1.000
2,O=C([O-])c1ccccc1-c1c2cc(Br)c(=O)c(Br)c-2oc2c(...,O,0.200
3,O=C([O-])c1ccccc1-c1c2cc(I)c(=O)c(I)c-2oc2c(I)...,O,0.020
4,O=C([O-])c1c(Cl)c(Cl)c(Cl)c(Cl)c1-c1c2cc(I)c(=...,O,0.018


In [8]:
desc_mol = torch.load("../torch_data/descriptors.pt")

C:\Users\kappa\AppData\Local\Temp\ipykernel_29556\2674240884.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  desc_mol = torch.load("../torch_data/descriptors.pt")


In [9]:
fgp_mol = torch.load("../torch_data/fingerprints.pt")

C:\Users\kappa\AppData\Local\Temp\ipykernel_29556\2761975527.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  fgp_mol = torch.load("../torch_data/fingerprints.pt")


In [10]:
chemB_mol = torch.tensor(chemB_mol)
chemB_sol = torch.tensor(chemB_sol)

C:\Users\kappa\AppData\Local\Temp\ipykernel_29556\732406262.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  chemB_mol = torch.tensor(chemB_mol)
C:\Users\kappa\AppData\Local\Temp\ipykernel_29556\732406262.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  chemB_sol = torch.tensor(chemB_sol)


### Index matching with the dataset to set aside 10% of already calculated embeddings

In [11]:
print(len(msl_new))
print(len(vecs_mols))
print(len(vecs_sols))
print(len(chemB_mol))
print(len(chemB_sol))
print(len(molformer_mol))
print(len(molformer_sol))

16648
16648
16648
16648
16648
16648
16648


In [12]:
uniq_mols = set(msl_new['Chromophore'])
print(len(uniq_mols))
print(uniq_mols)

7437
{'COc1ccc(/C=C2\\OC(=O)c3cc(N)ccc32)cc1', 'N#Cc1c(-n2c3ccccc3c3cc(I)ccc32)c(C#N)c(-n2c3ccccc3c3cc(I)ccc32)c(-n2c3ccccc3c3cc(I)ccc32)c1-n1c2ccccc2c2cc(I)ccc21', 'CC(C)(C)[Si](C)(C)C#CC1=CC2=c3cc(C#C[Si](C)(C)C(C)(C)C)c4nc(C(F)(F)C(F)(F)C(F)(F)F)nc5c(C#C[Si](C)(C)C(C)(C)C)cc(c3c54)C3=CC(C#C[Si](C)(C)C(C)(C)C)=C4N=C(C(F)(F)C(F)(F)C(F)(F)F)N=C1C4C32', 'NC(C=C1)=CC2=C1C3=[O+]C4=CC=CC=C4C(C5=CC=CC=C5C([O-])=O)=C3C2', 'CC(C)[Si](C#Cc1cc2c(C#C[Si](C(C)C)(C(C)C)C(C)C)c(C#C[Si](C(C)C)(C(C)C)C(C)C)c3cc(C#C[Si](C(C)C)(C(C)C)C(C)C)cc4c(C#C[Si](C(C)C)(C(C)C)C(C)C)c(C#C[Si](C(C)C)(C(C)C)C(C)C)c(c1)c2c34)(C(C)C)C(C)C', 'O=C1C2=CC=C(O)C=C2OC=C1C3=CC=C(OC)C=C3', 'C#CCOc1ccc(/C=C/C2=[N+]3C(=C(c4ccccc4)c4c(C)cc(C)n4[B-]3(F)F)C(C)=C2)cc1', 'CC1=CC(C)=[N+]2C1=C(c1ccccc1)c1c(C)cc(/C=C/c3ccc(N(C)C)cc3)n1[B-]2(F)F', 'O=C(O)C1=C/C(=C/C=[N+]2/c3cc(O)c(C[C@@H]4O[C@H](CO)[C@@H](O)C(O)[C@H]4O)cc3C[C@H]2C(=O)O)C[C@@H](C(=O)O)N1', 'COc1ccc(C#Cc2ccc3nc4ccc(C#Cc5ccc(OC)cc5)cn4c(=O)c3c2)cc1', 'NC(=O)CC[C@H](/[NH+]=

In [14]:
import random

random.seed(42)
uniq_10p = random.sample(list(uniq_mols), 744)
print(len(uniq_10p))
print(uniq_10p[:10])

744
['CN1/C(=C/C=N/c2sc3ccccc3[n+]2C)C(C)(C)c2ccccc21', 'O=C(C1=C2C(C3=CC=C24)=C(C5=C6C3=C(C7=CC=C(C(F)(F)F)C=C7)C=C8C6=C(C(N(CCCCCCCCCCCC)C8=O)=O)C=C5)C(C9=CC=C(C(F)(F)F)C=C9)=C1)N(CCCCCCCCCCCC)C4=O', 'C[Si](C)(C)C#CC(C#C[Si](C)(C)C)=Cc1ccc(C=C(C#C[Si](C)(C)C)C#C[Si](C)(C)C)cc1', 'Nc1cccc2cc3ccccc3cc12', 'O=C(C1=CC2=NC(C(O3)=CC=C3S(=O)(O)=O)=C(NC4CCCCC4)N2C=C1)OC', 'Cn1nc(-c2ccccc2)c2nc3cc(F)ccc3nc21', 'c1ccc2c(c1)Sc1ccc(-n3c4ccccc4c4ccccc43)cc1S2', 'CC(C)(C)c1ccc(-c2nnc(-c3c(-n4c5ccccc5c5ccccc54)c(-n4c5ccccc5c5ccccc54)c(-n4c5ccccc5c5ccccc54)c(-n4c5ccccc5c5ccccc54)c3-n3c4ccccc4c4ccccc43)o2)cc1', 'COc1cc(OC)cc(-c2nnc(-c3cccc(-c4nnc(-c5cc(OC)cc(OC)c5)o4)c3)o2)c1', 'N#Cc1ccc(/C=C2\\N=C(c3ccccc3)OC2=O)cc1']


In [15]:
with open('../input_data/partial/uniq_mols.text', 'w') as f:
    f.write('\n'.join(uniq_10p))

In [16]:
indices_10p = np.where(msl_new['Chromophore'].isin(uniq_10p))[0].tolist()
print(len(indices_10p))
print(indices_10p)

1565
[1, 25, 26, 27, 28, 30, 49, 59, 60, 61, 62, 203, 210, 217, 224, 247, 251, 275, 338, 339, 340, 341, 342, 343, 366, 368, 369, 370, 371, 372, 373, 374, 375, 376, 381, 383, 384, 390, 429, 462, 493, 494, 495, 496, 525, 535, 548, 550, 574, 598, 599, 600, 601, 602, 603, 678, 679, 680, 681, 682, 683, 684, 685, 721, 722, 725, 730, 767, 776, 786, 796, 834, 844, 853, 857, 893, 894, 895, 896, 897, 913, 924, 992, 1036, 1060, 1061, 1082, 1083, 1084, 1085, 1086, 1121, 1122, 1147, 1149, 1160, 1161, 1163, 1168, 1180, 1186, 1189, 1197, 1281, 1282, 1287, 1290, 1295, 1319, 1321, 1390, 1409, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454, 1494, 1508, 1510, 1543, 1545, 1546, 1547, 1549, 1551, 1552, 1553, 1574, 1628, 1636, 1741, 1749, 1757, 1765, 1855, 1863, 1866, 1869, 1875, 1880, 1901, 1912, 2022, 2023, 2024, 2025, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 2087, 2088, 2089, 2090, 2212, 2218, 2241, 2242, 2249, 2255, 2283, 2284, 2285, 2287, 2288, 2292, 2297, 2309, 2313, 231

In [17]:
msl_90p = msl_new.drop(index=indices_10p)
print(msl_90p.shape)
msl_90p.head()

(15083, 3)


,Chromophore,Solvent,Quantum yield
0,O=C([O-])c1ccccc1-c1c2ccc(=O)cc-2oc2cc([O-])ccc12,O,0.950
2,O=C([O-])c1ccccc1-c1c2cc(Br)c(=O)c(Br)c-2oc2c(...,O,0.200
3,O=C([O-])c1ccccc1-c1c2cc(I)c(=O)c(I)c-2oc2c(I)...,O,0.020
4,O=C([O-])c1c(Cl)c(Cl)c(Cl)c(Cl)c1-c1c2cc(I)c(=...,O,0.018
5,O=C([O-])c1ccccc1-c1c2cc(Br)c(=O)c(Br)c-2oc2c(...,CO,0.600


In [70]:
msl_10p = msl_new.iloc[indices_10p]
print(msl_10p.shape)
msl_10p.head()

(1565, 3)


,Chromophore,Solvent,Quantum yield
1,O=C([O-])c1ccccc1C1=c2cc3c4c(c2Oc2c1cc1c5c2CCC...,CO,1.00
25,CCC12C(=C3C(=C4c5ccc(-c6ccc(C)s6)cc5S(=O)(=O)C...,CCCCCC,0.72
26,CCC12C(=C3C(=C4c5ccc(-c6ccc(C)s6)cc5S(=O)(=O)C...,C1COCCO1,0.78
27,CCC12C(=C3C(=C4c5ccc(-c6ccc(C)s6)cc5S(=O)(=O)C...,CCO,0.62
28,CCC12C(=C3C(=C4c5ccc(-c6ccc(C)s6)cc5S(=O)(=O)C...,CC#N,0.67


In [29]:
msl_90p.to_csv('../input_data/msl_90p.csv', index=False)

In [72]:
msl_10p.to_csv('../input_data/msl_10p.csv', index=False)

In [18]:
'''
print(len(vecs_mols))
print(len(vecs_sols))
print(len(chemB_mol))
print(len(chemB_sol))
print(len(molformer_mol))
print(len(molformer_sol))
'''
indices_10p = torch.tensor(indices_10p)
indices_tt = torch.arange(vecs_mols.shape[0])

mask = ~torch.isin(indices_tt, indices_10p)
#print(mask)

vecs_mols90p = vecs_mols[mask]
vecs_sols90p = vecs_sols[mask]
chemB_mol90p = chemB_mol[mask]
chemB_sol90p = chemB_sol[mask]
molformer_mol90p = molformer_mol[mask]
molformer_sol90p = molformer_sol[mask]

print(vecs_mols90p.shape)
print(vecs_sols90p.shape)
print(chemB_mol90p.shape)
print(chemB_sol90p.shape)
print(molformer_mol90p.shape)
print(molformer_sol90p.shape)

torch.Size([15083, 256])
torch.Size([15083, 256])
torch.Size([15083, 768])
torch.Size([15083, 768])
torch.Size([15083, 768])
torch.Size([15083, 768])


In [22]:
fgp_mol90p = fgp_mol[mask]
desc_mol90p = desc_mol[mask]

print(fgp_mol90p.shape)
print(desc_mol90p.shape)

torch.Size([15083, 2048])
torch.Size([15083, 217])


In [24]:
torch.save(vecs_mols90p, "../torch_data/train_split/vecs_mols90p.pt")
torch.save(vecs_sols90p, "../torch_data/train_split/vecs_sols90p.pt")
torch.save(chemB_mol90p, "../torch_data/train_split/chemB_mol90p.pt")
torch.save(chemB_sol90p, "../torch_data/train_split/chemB_sol90p.pt")
torch.save(molformer_mol90p, "../torch_data/train_split/molformer_mol90p.pt")
torch.save(molformer_sol90p, "../torch_data/train_split/molformer_sol90p.pt")
torch.save(fgp_mol90p, "../torch_data/train_split/fgp_mol90p.pt")
torch.save(desc_mol90p, "../torch_data/train_split/desc_mol90p.pt")

In [25]:
test_mask = torch.isin(indices_tt, indices_10p)
print(test_mask)

tensor([False,  True, False,  ..., False, False, False])


In [26]:
vecs_mols10p = vecs_mols[test_mask]
vecs_sols10p = vecs_sols[test_mask]
chemB_mol10p = chemB_mol[test_mask]
chemB_sol10p = chemB_sol[test_mask]
molformer_mol10p = molformer_mol[test_mask]
molformer_sol10p = molformer_sol[test_mask]
fgp_mol10p = fgp_mol[test_mask]
desc_mol10p = desc_mol[test_mask]

print(vecs_mols10p.shape)
print(vecs_sols10p.shape)
print(chemB_mol10p.shape)
print(chemB_sol10p.shape)
print(molformer_mol10p.shape)
print(molformer_sol10p.shape)
print(fgp_mol10p.shape)
print(desc_mol10p.shape)

torch.Size([1565, 256])
torch.Size([1565, 256])
torch.Size([1565, 768])
torch.Size([1565, 768])
torch.Size([1565, 768])
torch.Size([1565, 768])
torch.Size([1565, 2048])
torch.Size([1565, 217])


In [27]:
torch.save(vecs_mols10p, "../torch_data/test_split/vecs_mols10p.pt")
torch.save(vecs_sols10p, "../torch_data/test_split/vecs_sols10p.pt")
torch.save(chemB_mol10p, "../torch_data/test_split/chemB_mol10p.pt")
torch.save(chemB_sol10p, "../torch_data/test_split/chemB_sol10p.pt")
torch.save(molformer_mol10p, "../torch_data/test_split/molformer_mol10p.pt")
torch.save(molformer_sol10p, "../torch_data/test_split/molformer_sol10p.pt")
torch.save(fgp_mol10p, "../torch_data/test_split/fgp_mol10p.pt")
torch.save(desc_mol10p, "../torch_data/test_split/desc_mol10p.pt")

### computing rdkit descs for sols and fingerprints for sols

In [30]:
from rdkit import Chem
from rdkit.Chem import Descriptors

def sol_descs(smile):
    sol = Chem.MolFromSmiles(smile)
    descs = []
    for _, func in Descriptors._descList:
        descs.append(func(sol))
    return descs

In [34]:
desc_df = []
for idx, smile in enumerate(msl_new['Solvent']):
    vals = sol_descs(smile)
    desc_df.append(vals)
    print(idx+1)

desc_df = pd.DataFrame(desc_df)
print(len(desc_df))
desc_df.head()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,216
0,0.0,0.0,0.0,0.0,0.327748,0.0,18.015,15.999,18.010565,8,...,0,0,0,0,0,0,0,0,0,0
1,7.0,7.0,1.0,1.0,0.385284,3.0,32.042,28.010,32.026215,14,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.327748,0.0,18.015,15.999,18.010565,8,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.327748,0.0,18.015,15.999,18.010565,8,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.327748,0.0,18.015,15.999,18.010565,8,...,0,0,0,0,0,0,0,0,0,0


In [37]:
scaler = StandardScaler()
desc_scale = scaler.fit_transform(desc_df.values)

desc_sol = torch.tensor(desc_scale, dtype=torch.float32).to(vecs_sols.device)
print(desc_sol.shape)

torch.Size([16648, 217])


In [38]:
torch.save(desc_sol, "../torch_data/descriptors_sols.pt")

In [39]:
from rdkit.Chem import rdFingerprintGenerator

fpgen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

In [40]:
def sol2fgprints(smiles_list):
    fgp_list = []
    for idx, smile in enumerate(smiles_list):
        sol = Chem.MolFromSmiles(smile)
        fgp = fpgen.GetCountFingerprint(sol)
        
        arr = np.zeros(2048, dtype=np.int32)
        for k, v in fgp.GetNonzeroElements().items():
            arr[k] = v

        fgp_list.append(arr)
        print(idx)

    return np.vstack(fgp_list)

In [41]:
fgp_np = sol2fgprints(msl_new['Solvent'])
print(len(fgp_np))
print(fgp_np.shape)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [43]:
fgp_sol = torch.tensor(fgp_np)
print(len(fgp_sol))
print(fgp_sol.shape)

16648
torch.Size([16648, 2048])


In [44]:
torch.save(fgp_sol, '../torch_data/fingerprints_sol.pt')

In [45]:
desc_sol90p = desc_sol[mask]
desc_sol10p = desc_sol[test_mask]

fgp_sol90p = fgp_sol[mask]
fgp_sol10p = fgp_sol[test_mask]

print(desc_sol90p.shape)
print(desc_sol10p.shape)
print(fgp_sol90p.shape)
print(fgp_sol10p.shape)

torch.Size([15083, 217])
torch.Size([1565, 217])
torch.Size([15083, 2048])
torch.Size([1565, 2048])


In [46]:
torch.save(desc_sol90p, "../torch_data/train_split/desc_sol90p.pt")
torch.save(desc_sol10p, "../torch_data/test_split/desc_sol10p.pt")

torch.save(fgp_sol90p, "../torch_data/train_split/fgp_sol90p.pt")
torch.save(fgp_sol10p, "../torch_data/test_split/fgp_sol10p.pt")

### split validation

In [47]:
desc_df = []
for idx, smile in enumerate(msl_90p['Chromophore']):
    vals = sol_descs(smile)
    desc_df.append(vals)
    print(idx+1)

desc_df = pd.DataFrame(desc_df)
print(len(desc_df))
desc_df.head()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,216
0,11.683581,11.683581,0.021944,-1.306869,0.524072,11.040000,330.295,320.215,330.053921,122,...,0,0,0,0,0,0,0,0,0,0
1,12.408729,12.408729,0.029454,-1.347366,0.284676,11.310345,645.879,639.831,641.695969,146,...,0,0,0,0,0,0,0,0,0,0
2,12.503623,12.503623,0.021975,-1.307991,0.220631,11.310345,833.879,827.831,833.640512,146,...,0,0,0,0,0,0,0,0,0,0
3,12.637713,12.637713,0.057996,-1.632189,0.093205,11.515152,971.659,969.643,969.484623,170,...,0,0,0,0,0,0,0,0,0,0
4,12.408729,12.408729,0.029454,-1.347366,0.284676,11.310345,645.879,639.831,641.695969,146,...,0,0,0,0,0,0,0,0,0,0


In [48]:
scaler = StandardScaler()
desc_scale = scaler.fit_transform(desc_df.values)

test_mol = torch.tensor(desc_scale, dtype=torch.float32).to(vecs_mols90p.device)
print(test_mol.shape)

torch.Size([15083, 217])


In [54]:
print(torch.isnan(desc_mol).any().item(), torch.isinf(desc_mol).any().item())
print(torch.isnan(desc_mol90p).any().item(), torch.isinf(desc_mol90p).any().item())
print(torch.allclose(desc_mol90p, desc_mol[mask], equal_nan=True))

True False
True False
True


In [55]:
test_fgp = sol2fgprints(msl_90p['Chromophore'])
test_fgp = torch.tensor(test_fgp)
print(len(test_fgp))
print(test_fgp.shape)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [56]:
print(torch.equal(test_fgp, fgp_mol90p))

True


In [57]:
import torch
from coati.models.io import load_e3gnn_smiles_clip_e2e

encoder, tokenizer = load_e3gnn_smiles_clip_e2e(
    freeze=True,
    device=torch.device("cuda:0"),
    doc_url="s3://terray-public/models/grande_closed.pkl",
)

Loading model from s3://terray-public/models/grande_closed.pkl
File downloaded successfully to .\models/grande_closed.pkl
Loading tokenizer may_closedparen from s3://terray-public/models/grande_closed.pkl
number of parameters: 12.64M
number of parameters Total: 2.44M xformer: 17.92M Total: 20.36M 
vocab_name not found in tokenizer_vocabs, trying to load from file
Freezing encoder
20561664 params frozen!


In [58]:
from coati.generative.coati_purifications import embed_smiles

test_mols = []   

for idx, smile in enumerate(msl_90p['Chromophore'][:100]):
    frag_vecs = []
    if '.' in smile:
        fragments = smile.split('.')
        for frag in fragments:
            vec = embed_smiles(frag, encoder, tokenizer)
            frag_vecs.append(vec)
        
        if len(frag_vecs) > 0:
            fused_emb = torch.mean(torch.stack(frag_vecs), dim=0)
            test_mols.append(fused_emb)
    else:
        vec = embed_smiles(smile, encoder, tokenizer)
        test_mols.append(vec)

    print(idx+1)

print(len(test_mols))
test_mols = torch.stack(test_mols)
print(test_mols.shape)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
100
torch.Size([100, 256])


In [63]:
test_mols = test_mols.to(vecs_mols90p.device)
print(torch.equal(test_mols, vecs_mols90p[:100]))

True


### recomputing descriptors for mol and sol only on 90% split and scaling both train and test

In [79]:
from rdkit import Chem
from rdkit.Chem import Descriptors

def comp_descs(smile):
    sm = Chem.MolFromSmiles(smile)
    descs = []
    for _, func in Descriptors._descList:
        descs.append(func(sm))
    return descs

In [80]:
new90p_descs_m = []
for idx, smile in enumerate(msl_90p['Chromophore']):
    vals = comp_descs(smile)
    new90p_descs_m.append(vals)
    print(idx+1)

new90p_descs_m = pd.DataFrame(new90p_descs_m)
print(len(new90p_descs_m))
new90p_descs_m.head()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,216
0,11.683581,11.683581,0.021944,-1.306869,0.524072,11.040000,330.295,320.215,330.053921,122,...,0,0,0,0,0,0,0,0,0,0
1,12.408729,12.408729,0.029454,-1.347366,0.284676,11.310345,645.879,639.831,641.695969,146,...,0,0,0,0,0,0,0,0,0,0
2,12.503623,12.503623,0.021975,-1.307991,0.220631,11.310345,833.879,827.831,833.640512,146,...,0,0,0,0,0,0,0,0,0,0
3,12.637713,12.637713,0.057996,-1.632189,0.093205,11.515152,971.659,969.643,969.484623,170,...,0,0,0,0,0,0,0,0,0,0
4,12.408729,12.408729,0.029454,-1.347366,0.284676,11.310345,645.879,639.831,641.695969,146,...,0,0,0,0,0,0,0,0,0,0


In [67]:
msl_90p = pd.read_csv('../input_data/msl_90p.csv')

In [81]:
new90p_descs_s = []
for idx, smile in enumerate(msl_90p['Solvent']):
    vals = comp_descs(smile)
    new90p_descs_s.append(vals)
    print(idx+1)

new90p_descs_s = pd.DataFrame(new90p_descs_s)
print(len(new90p_descs_s))
new90p_descs_s.head()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,216
0,0.0,0.0,0.0,0.0,0.327748,0.0,18.015,15.999,18.010565,8,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.327748,0.0,18.015,15.999,18.010565,8,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.327748,0.0,18.015,15.999,18.010565,8,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,0.327748,0.0,18.015,15.999,18.010565,8,...,0,0,0,0,0,0,0,0,0,0
4,7.0,7.0,1.0,1.0,0.385284,3.0,32.042,28.010,32.026215,14,...,0,0,0,0,0,0,0,0,0,0


In [82]:
new10p_descs_m = []
for idx, smile in enumerate(msl_10p['Chromophore']):
    vals = comp_descs(smile)
    new10p_descs_m.append(vals)
    print(idx+1)

new10p_descs_m = pd.DataFrame(new10p_descs_m)
print(len(new10p_descs_m))
new10p_descs_m.head()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,216
0,12.335178,12.335178,0.247484,-1.130887,0.405433,18.864865,490.603,460.363,490.225643,186,...,0,0,0,0,0,0,0,0,0,0
1,16.269954,16.269954,0.358490,-6.004095,0.194888,27.469388,752.845,726.637,752.061813,256,...,0,0,2,0,0,0,0,2,0,0
2,16.269954,16.269954,0.358490,-6.004095,0.194888,27.469388,752.845,726.637,752.061813,256,...,0,0,2,0,0,0,0,2,0,0
3,16.269954,16.269954,0.358490,-6.004095,0.194888,27.469388,752.845,726.637,752.061813,256,...,0,0,2,0,0,0,0,2,0,0
4,16.269954,16.269954,0.358490,-6.004095,0.194888,27.469388,752.845,726.637,752.061813,256,...,0,0,2,0,0,0,0,2,0,0


In [83]:
new10p_descs_s = []
for idx, smile in enumerate(msl_10p['Solvent']):
    vals = comp_descs(smile)
    new10p_descs_s.append(vals)
    print(idx+1)

new10p_descs_s = pd.DataFrame(new10p_descs_s)
print(len(new10p_descs_s))
new10p_descs_s.head()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,216
0,7.000000,7.000000,1.000000,1.000000,0.385284,3.000000,32.042,28.010,32.026215,14,...,0,0,0,0,0,0,0,0,0,0
1,2.231806,2.231806,1.355000,1.355000,0.462954,9.000000,86.178,72.066,86.109550,38,...,0,0,0,0,0,0,0,0,3,0
2,4.944444,4.944444,0.777778,0.777778,0.415749,24.000000,88.106,80.042,88.052429,36,...,0,0,0,0,0,0,0,0,0,0
3,7.569444,7.569444,0.250000,0.250000,0.406808,6.000000,46.069,40.021,46.041865,20,...,0,0,0,0,0,0,0,0,0,0
4,7.319444,7.319444,1.430556,1.430556,0.386981,2.666667,41.053,38.029,41.026549,16,...,0,0,0,0,0,0,0,0,0,0


In [84]:
print(type(new90p_descs_m))
print(type(new90p_descs_s))
print(type(new10p_descs_m))
print(type(new10p_descs_s))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [85]:
imp_m = SimpleImputer(strategy="median")
imp_s = SimpleImputer(strategy="median")

X90_m_imp = imp_m.fit_transform(new90p_descs_m.values)
X90_s_imp = imp_s.fit_transform(new90p_descs_s.values)

sc_m = StandardScaler()
sc_s = StandardScaler()

X90_m_sc = sc_m.fit_transform(X90_m_imp)
X90_s_sc = sc_s.fit_transform(X90_s_imp)

print(X90_m_sc.shape, X90_s_sc.shape)

(15083, 217) (15083, 217)


In [86]:
X10_m_imp = imp_m.transform(new10p_descs_m.values)
X10_s_imp = imp_s.transform(new10p_descs_s.values)

X10_m_sc = sc_m.transform(X10_m_imp)
X10_s_sc = sc_s.transform(X10_s_imp)

print(X10_m_sc.shape, X10_s_sc.shape)

(1565, 217) (1565, 217)


In [87]:
torch.save(X90_m_sc, "../torch_data/train_split/desc_mol90p.pt")
torch.save(X90_s_sc, "../torch_data/train_split/desc_sol90p.pt")

torch.save(X10_m_sc, "../torch_data/test_split/desc_mol10p.pt")
torch.save(X10_s_sc, "../torch_data/test_split/desc_sol10p.pt")